In [218]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, NormalPredictor, KNNBasic
from surprise.model_selection import cross_validate
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import sparse
from sklearn.metrics.pairwise import *
from surprise import Reader, Dataset, KNNBasic, accuracy
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold

In [173]:
df = pd.read_csv('radiate_crushes.csv', parse_dates=['created_on','updated_on'])

In [174]:
df['is_positive'] = df['is_positive'].map({True:1, False:-1})

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80698314 entries, 80698313 to 0
Data columns (total 6 columns):
peep_to_id      int64
peep_from_id    int64
bubble_id       float64
is_positive     int64
created_on      datetime64[ns]
updated_on      datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(3)
memory usage: 4.2 GB


In [176]:
df.sort_values(by='updated_on', inplace=True)

In [177]:
df.head()

,peep_to_id,peep_from_id,bubble_id,is_positive,created_on,updated_on
80698313,200016,200017,10.0,1,2015-05-03 05:25:16.576871,2015-05-24 03:45:03.551374
80698161,200007,200043,9.0,1,2015-05-20 03:29:34.081895,2015-05-24 03:45:03.551374
80698160,200006,200045,9.0,-1,2015-05-21 14:31:45.561990,2015-05-24 03:45:03.551374
80698159,200043,200007,9.0,1,2015-05-20 03:34:11.161917,2015-05-24 03:45:03.551374
80698158,200040,200007,9.0,1,2015-05-20 03:33:58.633158,2015-05-24 03:45:03.551374


In [191]:
df.dropna(inplace=True)

In [178]:
df['is_positive'].value_counts()

 1    46174012
-1    34524302
Name: is_positive, dtype: int64

In [181]:
# remove duplicates

df2 = df.groupby(['peep_to_id','peep_from_id']).last()

In [182]:
df2.reset_index(inplace=True)

In [184]:
swipe_counts = df2.groupby('peep_from_id',as_index=False)['peep_to_id'].count()
swipe_counts.rename(columns={'peep_to_id':'num_swipes'}, inplace=True)

In [186]:
df2 = df2.merge(swipe_counts)

In [187]:
df2 = df2[df2['num_swipes']>=25]

# decide later if we should keep this

In [209]:
# df2.to_pickle('df2.pkl')

In [211]:
df2 = df2[['peep_from_id','peep_to_id','is_positive']]

In [212]:
df2.head()

,peep_from_id,peep_to_id,is_positive
0,200007,200006,-1
1,200007,200008,-1
2,200007,200013,1
3,200007,200016,1
4,200007,200017,-1


In [194]:
small_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80000 entries, 0 to 153793
Data columns (total 3 columns):
peep_to_id      80000 non-null int64
peep_from_id    80000 non-null int64
is_positive     80000 non-null int64
dtypes: int64(3)
memory usage: 2.4 MB


In [10]:
# plt.hist(df['num_swipes'], range=(0,100))
# plt.show()

In [193]:
small_df = df2[['peep_to_id', 'peep_from_id', 'is_positive']].iloc[:80000,:]

In [148]:
# utility_mat = sparse.coo_matrix((small_df['is_positive'], (small_df['peep_from_id'], small_df['peep_to_id'])))

In [195]:
reader = Reader(rating_scale=(-1,1))

In [219]:
data_small = Dataset.load_from_df(small_df[['peep_from_id','peep_to_id','is_positive']], reader)

In [220]:
trainset, testset = train_test_split(data_small, test_size=.25)

In [221]:
algo = KNNBasic()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0147


1.0146890425226514

# CLEANER CODE

In [222]:
df2.head()

,peep_from_id,peep_to_id,is_positive
0,200007,200006,-1
1,200007,200008,-1
2,200007,200013,1
3,200007,200016,1
4,200007,200017,-1


In [224]:
reader = Reader(rating_scale=(-1,1))
data = Dataset.load_from_df(df2, reader)

In [ ]:
trainset, testset = train_test_split(data, test_size=.25)

knn = KNNBasic()
knn.fit(trainset)
predictions = knn.test(testset)

accuracy.rmse(predictions)

In [197]:
# kf = KFold(n_splits=3)
# knn = KNNBasic()

# for trainset, testset in kf.split(data):

#     knn.fit(trainset)
#     predictions = knn.test(testset)

#     accuracy.rmse(predictions, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0192
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0279
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 1.0208


In [207]:
predictions[:20]

[Prediction(uid=201166, iid=203650, r_ui=-1.0, est=-1, details={u'actual_k': 1, u'was_impossible': False}),
 Prediction(uid=201524, iid=215271, r_ui=-1.0, est=-1, details={u'actual_k': 4, u'was_impossible': False}),
 Prediction(uid=201162, iid=288093, r_ui=-1.0, est=1, details={u'actual_k': 1, u'was_impossible': False}),
 Prediction(uid=200180, iid=200817, r_ui=1.0, est=0.031020853279808858, details={u'actual_k': 15, u'was_impossible': False}),
 Prediction(uid=200017, iid=203964, r_ui=1.0, est=-1, details={u'actual_k': 2, u'was_impossible': False}),
 Prediction(uid=201490, iid=220042, r_ui=1.0, est=-0.07968650391870091, details={u'reason': 'User and/or item is unkown.', u'was_impossible': True}),
 Prediction(uid=200629, iid=267772, r_ui=-1.0, est=-0.07968650391870091, details={u'reason': 'User and/or item is unkown.', u'was_impossible': True}),
 Prediction(uid=200016, iid=288400, r_ui=-1.0, est=-0.07968650391870091, details={u'reason': 'User and/or item is unkown.', u'was_impossible': 

In [208]:
small_df[(small_df['peep_from_id']==201490) & (small_df['peep_to_id']==220042)]

,peep_to_id,peep_from_id,is_positive
121170,220042,201490,1


In [201]:
# similarities = cosine_similarity(utility_mat)
# sims = paired_cosine_distances(sparse_df[])

In [129]:
# sparse_df = pd.DataFrame({'index': utility_mat.row, 'col': utility_mat.col, 'data': utility_mat.data})

In [202]:
# sparse_df = pd.DataFrame({'index': utility_mat.row, 'col': utility_mat.col, 'data': utlity_mat.data})

In [204]:
knn = KNNBasic(name='cosine', user_based=True)

In [206]:
# train, test = train_test_split(small_df.)
# knn.train(utility_mat, n_users=len(utility_mat))